In [80]:
import requests
import json
import os
import climate
from pprint import pprint
import pandas as pd
import numpy as np
import io
import census
from pprint import pprint

In [81]:
cen = census.CensusData()
cli = climate.ClimateAgent()
pgpw = os.getenv("POSTGRES_PASSWORD") 
server, engine = cen.connect_to_postgres(password=pgpw) #need to add create=True if


In [85]:
query = """
SELECT *
FROM city_data
"""
list(pd.read_sql_query(query, con=engine).columns)

['city',
 'year',
 'male_worker_earnings',
 'female_worker_earnings',
 'pct_below_pov',
 'num_housing_units',
 'num_married_housing_units',
 'avg_household_size',
 'lt_10k_income',
 'gt_10k_lt_15k_income',
 'gt_15k_lt_25k_income',
 'gt_25k_lt_35k_income',
 'gt_35k_lt_45k_income',
 'gt_50k_lt_75k_income',
 'gt_75k_lt_100k_income',
 'gt_100k_lt_150k_income',
 'gt_150k_lt_200k_income',
 'gt_200k_income',
 'men_not_married',
 'married_but_separated',
 'divorced',
 'est_in_preschool',
 'est_in_kindergarten',
 'est_in_1_8',
 'est_in_9_12',
 'est_in_college',
 'pct_lt_ninth_grade',
 'pct_some_high_school',
 'pct_high_school',
 'pct_some_college',
 'pct_associates',
 'pct_bachelors',
 'pct_graduate',
 'unemployment_rate',
 'pct_in_ag_ind',
 'pct_in_construction',
 'pct_in_manufacturing',
 'pct_in_wholesale_trade',
 'pct_in_retail',
 'pct_in_transportation',
 'pct_in_information',
 'pct_in_finance',
 'pct_in_science',
 'pct_in_education',
 'pct_in_arts',
 'pct_in_pub_ad',
 'state',
 'county']

In [ ]:
station_id_one = cli.get_station_id("norfolk, virginia")
station_id_two = cli.get_station_id("augusta, virginia")

df_one = cli.get_climate_data_from_postgres(station_id_one, engine=engine)
df_two = cli.get_climate_data_from_postgres(station_id_two, engine=engine)

full_df = pd.concat([df_one, df_two])
full_df['date'] = pd.to_datetime(full_df['date'])
full_df['year'] = full_df['date'].dt.year
full_df['month'] = full_df['date'].dt.month
full_df.groupby(["id", "year"]).agg(
    {"prcp": "mean",
     "tmax": "mean",
     "tmin": "mean"}
)




prcp       tmax       tmin
id          year                                
USC00448062 1940  0.125246  64.136986  40.571038
            1941  0.063003  68.254190  40.819444
            1942  0.126686  66.779614  41.532967
            1943  0.091151  67.672176  41.791781
            1944  0.113297  67.964286  42.691257
...                    ...        ...        ...
USW00013737 2019  0.126521  71.594521  55.191781
            2020  0.153907  72.139344  55.871585
            2021  0.106630  70.717808  53.983562
            2022  0.107425  69.764384  52.906849
            2023  0.138493  70.865753  54.610959

[162 rows x 3 columns]

In [ ]:
headers = cli.make_headers()
station = cli.get_station_id("norfolk, virginia")
params = {"dataset":"global-summary-of-the-month",
          "stations": station,
          "startDate": "2022-01-01",
          "endDate": "2022-01-02"
          }
r = requests.get("https://www.ncei.noaa.gov/access/services/search/v1/data/", params=params, headers=headers)
data_types = json.loads(r.text)['results'][0]['dataTypes']
data_types    

In [63]:
query = """
SELECT *
FROM climate_data
LIMIT 1
"""
df = pd.read_sql_query(query, con=engine)
cols = list(df.columns)

for c in cols:
    print(c)

id
date
prcp
tmax
tmin


In [58]:
query = f"""
SELECT *
FROM city_data
WHERE station == '12345'
"""
record = pd.read_sql_query(query, con=engine)

ProgrammingError: (psycopg.errors.UndefinedColumn) column "station" does not exist
LINE 4: WHERE station == '12345'
              ^
HINT:  Perhaps you meant to reference the column "city_data.state".
[SQL: 
SELECT *
FROM city_data
WHERE station == '12345'
]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [ ]:
# ca.upload_city_data_to_postgres(tmp_df, engine=engine)
# # CITY YEAR will be the primary key(s)
# # all other data will follow after that

In [5]:
# to delete a table,,, weird syntax
from sqlalchemy import MetaData, Table
metadata = MetaData()

# Reflect the existing table
table_name = 'climate_data'
table = Table(table_name, metadata, autoload_with=engine)

# Drop the table
table.drop(engine)

In [ ]:
# query = """
# SELECT * 
# FROM cities
# """

# counties = list(pd.read_sql_query(query, con=engine)['county'])

# final_df = pd.DataFrame()
# for county in counties:
#     print("Working on: ", county)
#     city_data_df = ca.get_data_for_city(city=None, years=(2009,2024), county=county)
#     final_df = pd.concat([final_df, city_data_df], ignore_index=True)


In [ ]:
# ca.upload_city_data_to_postgres(final_df, engine=engine)

In [ ]:
# # existing_data = pd.read_sql('SELECT city, year FROM city_data', engine)
# # new_rows = final_df[~final_df[['city', 'year']].isin(existing_data[['city', 'year']])]
# # final_df[['city', 'year']].isin(existing_data[['city', 'year']])
# query = """
# SELECT * FROM city_data
# """

# check_df = pd.read_sql_query(query, con=engine)
# check_df
# #rows_to_upload = final_df[]
# # final_df.to_sql("city_data", con=engine, index=False, chunksize=1000, if_exists="replace")

In [ ]:

# query = """
# SELECT * FROM city_data
# """

# df = pd.read_sql_query(query, con=engine)
# df

In [ ]:
# y = "2009"
# url = "https://api.census.gov/data/" + f"{y}/acs/acs1/profile"

# get_metrics = "".join(x + "," for x in ca.list_of_metrics.keys())
# get_metrics = get_metrics[:len(get_metrics)-1] # get rid of the last comma that I appended

# params = {"key": os.getenv("CENSUS_KEY"),
#                     "get": get_metrics,
#                     "for": f"county:810",
#                     "in": "state:51"}

# r = requests.get(url=url, params=params)
# values = list(json.loads(r.text))
# pd.DataFrame(values[1:], columns=values[0])

In [ ]:
# ca.get_all_cities()

In [ ]:
# from html.parser import HTMLParser
# #census_url = "https://api.census.gov/data"
# year = "2023"
# dataset = "acs"
# url = f"https://api.census.gov/data/{year}/acs/acs1/profile"


# """
#     # https://api.census.gov/data/2022/acs/acs1/profile/variables.html 

#     # ones worth using:

#     # economic
#     DP03_0093E = estimate for earnings for first year male workers 
#     DP03_0094E = estimate for earnings for first year female workers
#     DP03_0119PE = estimated percentage of people below the poverty line
#     DP04_0001E = estimated housing units
#     DP02_0002E = estimated number of married housing units
#     DP02_0016E = estimated average household size

#     DP03_0052E: estimated total households less than $10,000 income
#     DP03_0053E: estimated total households between $10-15k
#     DP03_0054E: estimated total households between $15-25k
#     DP03_0055E: 25-35k
#     DP03_0056E: 35-45k
#     DP03_0057E: 50-75k
#     DP03_0058E: 75-100k
#     DP03_0059E: 100-150k
#     DP03_0060E: 150-200k
#     DP03_0061E: > 200k

#     # relationships
#     DP02_0026E = men over 15 y/o who have not married
#     DP02_0028E = men over 15 y/o who married, but separated
#     DP02_0030E = men over 15 y/o who are divorced
    
#     # education (currently)
#     DP02_0054E = estimated population of kids in nursery school/preschool
#     DP02_0055E = estimated population of kids in kindergarten
#     DP02_0056E = estimated population of kids in grades 1-8
#     DP02_0057E = estimated population of kids in grades 9-12
#     DP02_0058E = estimated population of kids in college/grad school

#     # education attainment
#     DP02_0060PE = estimated population of 25 years and older who have less than 9th grade education
#     DP02_0061PE = estimated population with some high school education
#     DP02_0062PE = estimated population with high school diploma
#     DP02_0063PE = estimated population with some college, no diploma
#     DP02_0064PE = estimated population with associates degrees
#     DP02_0065PE  = estimated population with bachelor's degree
#     DP02_0066PE = estimated population with graduate degrees

#     # unemployement rate
#     DP03_0009PE

#     # industry
#     DP03_0033PE: agriculture, forestry, fishing, hunting, mining
#     DP03_0034PE: construction
#     DP03_0035PE: manufacturing
#     DP03_0036PE: wholesale trade
#     DP03_0037PE: retail trade
#     DP03_0038PE: transportation, warehousing, utilities
#     DP03_0039PE: information
#     DP03_0040PE: finance, insurance, real estate, rental, leasing
#     DP03_0041PE: professional, scientific, administrative, waste management
#     DP03_0042PE: educational services/health care/ social assistance
#     DP03_0043PE: arts, entertainment, recreation, accommodation, food services
#     DP03_0045PE: public administration

# """
# variables = "NAME,DP02_0060PE,DP02_0061PE,DP02_0062PE,DP02_0063PE,DP02_0064PE,DP02_0065PE,DP02_0066PE"
# print(url)
# params = {"key": os.getenv("CENSUS_KEY"),
#            "for": "county:*", 
#            "get": variables,
#            "in": "state:51"
#           }
# r = requests.get(url, params=params)
# values = list(json.loads(r.text))

# df = pd.DataFrame(values[1:], columns=values[0])
# # df = df.rename(columns={
# #     "DP02_0060PE":"<9th",
# #     "DP02_0061PE": "some high school",
# #     "DP02_0062PE": "high school",
# #     "DP02_0063PE": "some college",
# #     "DP02_0064PE": "associates",
# #     "DP02_0065PE": "bachelors",
# #     "DP02_0066PE": "graduate"
# # })
# df

In [ ]:
co.upload_data_to_postgres(climate_data=climate_df, engine=engine)

In [ ]:
query = """
SELECT *
FROM climate_data
"""

pd.read_sql_query(query, con=engine)

In [ ]:
cities = cen.get_all_cities()
cities = list(cities['city'])

city_to_station = {}

full_df = pd.DataFrame()
for c in cities:
    if c == "fauquier":
        c = "front royal"
    if c == "jamescity":
        c = "williamsburg"
    if c == "princewilliam":
        c = "manassas"
    if c == "spotsylvania":
        c = "fredericksburg"

    if c == "beach":
        c = "Virginia Beach"
    c = c + ", virginia"
    print("Working on:", c)
    station_id = cli.get_station_id(c)
    climate_df = cli.get_data_for_station_id(station_id=station_id, start_date="1940-01-01", end_date="2023-12-31")
    full_df = pd.concat([full_df, climate_df], ignore_index=True)

In [ ]:
station_id = cli.get_station_id("virginia beach, virginia")
climate_df = cli.get_data_for_station_id(station_id=station_id, start_date="1940-01-01", end_date="2023-12-31")
full_df = pd.concat([full_df, climate_df], ignore_index=True)

In [ ]:
full_df = full_df.rename(columns={
    "STATION": "id",
    "DATE": "date",
    "PRCP": "prcp",
    "TMAX": "tmax",
    "TMIN": "tmin"
})


cli.upload_data_to_postgres(full_df, engine=engine)

Uploading city climate data to the database
city_data table does not exist yet, creating now with first command
Finished uploading city data to database


In [10]:
query = """
SELECT *
FROM climate_data
WHERE id='USW00093736'
"""
pd.read_sql_query(query, con=engine)

,id,date,prcp,tmax,tmin
0,USW00093736,1955-09-14,NaN,77.0,47.0
1,USW00093736,1955-09-15,NaN,88.0,55.0
2,USW00093736,1955-09-16,NaN,92.0,64.0
3,USW00093736,1955-09-17,NaN,80.0,70.0
4,USW00093736,1955-09-18,NaN,81.0,67.0
...,...,...,...,...,...
24879,USW00093736,2023-12-27,0.28,55.0,50.0
24880,USW00093736,2023-12-28,0.00,61.0,41.0
24881,USW00093736,2023-12-29,0.02,50.0,36.0
24882,USW00093736,2023-12-30,0.00,51.0,34.0


In [ ]:
col_one = city_to_station.keys()
col_two = city_to_station.values()

new_dicts = {"city": col_one, "id": col_two}

city_id_df = pd.DataFrame.from_dict(new_dicts)

In [ ]:
cli.upload_cities_and_ids(city_id_df, engine=engine)

In [ ]:
lat, long = co.get_lat_long(location="Fredericksburg VA")
bb = co.get_bounding_box(latitude=lat, longitude=long, buffer = 10)
bb

In [ ]:
bounding_box = f"{bb[0]},{bb[1]},{bb[2]},{bb[3]}"
print(bounding_box)
params = {"dataset":"global-summary-of-the-month",
                  "bbox": bounding_box,
                  "dataTypes": "TMIN,TMAX,PRCP",
                  "startDate": "2022-01-01",
                  "endDate": "2022-02-01"}
headers = co.make_headers()
base = "https://www.ncei.noaa.gov/access/services/search/v1/data"
r = requests.get(url=base, params=params, headers=headers)
r.text

In [ ]:
city_to_station

In [ ]:
final_df = pd.DataFrame()
for station in city_to_station.values():
    co.get_data_for_station_id(station)